# Text preprocessing - Pipeline

This code will preprocess a pair of documents in order to analyse their similarity afterwards.

Import libraries

In [120]:
import pandas as pd
import json
import nltk
#nltk.download()

Read files

In [121]:
pd.set_option('display.max_colwidth', -1)
## dfSourceV2 = pd.read_csv('../Dados/v2_semeval-2022_task8_train-data_batch.csv')
trainv1 = pd.read_csv('dados/train v0.1.csv')

trainv1_enen = trainv1[(trainv1['url1_lang']=='en') & (trainv1['url2_lang']=='en')]

trainv1_enen.head()

C:\Users\lucas\AppData\Local\Temp/ipykernel_4412/4022916483.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,https://web.archive.org/web/www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://web.archive.org/web/www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,https://web.archive.org/web/www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://web.archive.org/web/www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,https://web.archive.org/web/gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://web.archive.org/web/gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,https://web.archive.org/web/news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://web.archive.org/web/www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [122]:
trainv1[trainv1['pair_id']=='1568535170_1558667669']

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
2938,ar,ar,1568535170_1558667669,https://www.alarabiya.net/ar/aswaq/oil-and-gas/2020/04/04/%D8%AA%D8%B1%D9%85%D8%A8-%D9%8A%D9%84%D8%AA%D9%82%D9%8A-%D8%B1%D8%A4%D8%B3%D8%A7%D8%A1-%D8%B4%D8%B1%D9%83%D8%A7%D8%AA-%D8%A7%D9%84%D9%86%D9%81%D8%B7-%D9%81%D9%85%D8%A7%D8%B0%D8%A7-%D8%B3%D9%8A%D9%82%D8%AF%D9%85%D8%9F.html,https://www.khaberni.com/news/339402,https://web.archive.org/web/www.alarabiya.net/ar/aswaq/oil-and-gas/2020/04/04/%D8%AA%D8%B1%D9%85%D8%A8-%D9%8A%D9%84%D8%AA%D9%82%D9%8A-%D8%B1%D8%A4%D8%B3%D8%A7%D8%A1-%D8%B4%D8%B1%D9%83%D8%A7%D8%AA-%D8%A7%D9%84%D9%86%D9%81%D8%B7-%D9%81%D9%85%D8%A7%D8%B0%D8%A7-%D8%B3%D9%8A%D9%82%D8%AF%D9%85%D8%9F.html,https://web.archive.org/web/www.khaberni.com/news/339402,3.0,2.0,2.0,2.0,3.0,1.0,1.0


In [123]:
trainv1_enen[trainv1_enen['pair_id']=='1568535170_1558667669']

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone


### Carregar noticias a partir dos ids:

Functions that read json documents

In [124]:
#Function reads a json file
def readJsonFile(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data

    
#Function returns a dataframe with the text of the pairs
def getJsonDocumentPair(dataPath, pairId):
    listIds = pairId.split('_')
    doc1Id = listIds[0]
    doc2Id = listIds[1]

    doc1Path = dataPath + doc1Id[-2:] + '/' + doc1Id + '.json' 
    doc2Path = dataPath + doc2Id[-2:] + '/' + doc2Id + '.json' 

    doc1Json = readJsonFile(doc1Path)
    doc2Json = readJsonFile(doc2Path)

    return (doc1Json, doc2Json)

Getting text to preprocess

In [125]:
dataPath = 'dados/train v0.1/'
#jsonPair = getJsonDocumentPair(dataPath,'1484084337_1484110209')
#textDoc1 = jsonPair[0]['text']

lista_docs = []
values = trainv1_enen[['pair_id', 'Overall']]
lista_error = []
for index, values in values.iterrows():
    try:
        jsonPair = getJsonDocumentPair(dataPath, values['pair_id'])
        textDoc1 = jsonPair[0]['text']
        textDoc2 = jsonPair[1]['text']
        lista_docs.append((id, textDoc1, textDoc2, values['Overall']))
    except:
        lista_error.append(id)

#Creating DF to text
#dfText = pd.DataFrame([[textDoc1]], columns=['original_text'])
dfText = pd.DataFrame(lista_docs,  columns=['pair_id', 'doc1', 'doc2', 'Overall'])
dfText.head(1)

,pair_id,doc1,doc2,Overall
0,1568535170_1558667669,"MARTINSBURG, W.Va. — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia, police said.\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County, The Journal reported.\n\nCody Wade Braithwaite, 32, of Winchester, Virginia, was charged with DUI with death, fleeing DUI, and being a prohibited person in possession of a firearm and a fugitive from justice, Berkeley County Sheriff Curtis Keller said.\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop, but the suspect’s vehicle sped up and hit a car, which then struck a guardrail. The driver of the car was pronounced dead at the scene. The driver’s name was not immediately released.\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped, Keller said.\n\nBraithwaite was being held without bond in the Eastern Regional Jail. It wasn’t immediately known whether Braithwaite has an attorney.\n\n___\n\nInformation from: The Journal, http://journal-news.net/\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.","PORT-AU-PRINCE, Haiti — Haitian President Jovenel Moïse broke with tradition on Wednesday and celebrated the country’s independence day in the capital for security reasons following months of political turmoil.\n\nMoïse, whose government has been accused of corruption, denounced graft during his speech at the National Palace in Port-au-Prince and urged Haiti’s elite to work with the government and help create employment.\n\n“We’re still extremely poor,” he said. “Those who continue to get rich find it normal that they do not pay taxes, find it normal that there can be no competition, find it normal that they set prices for consumers, especially when this consumer is the state itself.”\n\nMoïse also apologized for the country’s ongoing power outages and renewed his 2016 campaign pledge to provide electricity 24 hours a day, saying it was harder to accomplish than he imagined.\n\nThe speech that marked the 216th anniversary of the world’s first black republic was originally slated to take place in the northern coastal town of Gonaives, where Jean-Jacques Dessalines declared Haiti’s independence. But the town, like many others, was hit by violent protests that began in September amid anger over corruption, fuel shortages and dwindling food supplies as opposition leaders and supporters demanded the resignation of Moïse. More than 40 people have been killed and dozens injured.\n\nLarge-scale protests in Port-au-Prince have since dissipated, although smaller ones are still occurring elsewhere in the country. On Wednesday, opposition leaders and supporters gathered in Gonaives to attend the funeral of an anti-government protester and then carried his coffin through the streets as more protesters joined them.\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.",4.0


# Inicio do pre-processamento

Removing punctuation

In [126]:
import string
string.punctuation
other_punctuation = '—“”'  

In [127]:
#Function that removes punctuation 
def removePunctuation(text):
    punctuationFreeDoc = "".join([i for i in text if i not in string.punctuation+other_punctuation])
    return punctuationFreeDoc


#Storing the puntuation free text
dfText['clean_msg1']= dfText['doc1'].apply(lambda x:removePunctuation(x))
dfText['clean_msg2']= dfText['doc2'].apply(lambda x:removePunctuation(x))
dfText.head(1)

,pair_id,doc1,doc2,Overall,clean_msg1,clean_msg2
0,1568535170_1558667669,"MARTINSBURG, W.Va. — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia, police said.\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County, The Journal reported.\n\nCody Wade Braithwaite, 32, of Winchester, Virginia, was charged with DUI with death, fleeing DUI, and being a prohibited person in possession of a firearm and a fugitive from justice, Berkeley County Sheriff Curtis Keller said.\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop, but the suspect’s vehicle sped up and hit a car, which then struck a guardrail. The driver of the car was pronounced dead at the scene. The driver’s name was not immediately released.\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped, Keller said.\n\nBraithwaite was being held without bond in the Eastern Regional Jail. It wasn’t immediately known whether Braithwaite has an attorney.\n\n___\n\nInformation from: The Journal, http://journal-news.net/\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.","PORT-AU-PRINCE, Haiti — Haitian President Jovenel Moïse broke with tradition on Wednesday and celebrated the country’s independence day in the capital for security reasons following months of political turmoil.\n\nMoïse, whose government has been accused of corruption, denounced graft during his speech at the National Palace in Port-au-Prince and urged Haiti’s elite to work with the government and help create employment.\n\n“We’re still extremely poor,” he said. “Those who continue to get rich find it normal that they do not pay taxes, find it normal that there can be no competition, find it normal that they set prices for consumers, especially when this consumer is the state itself.”\n\nMoïse also apologized for the country’s ongoing power outages and renewed his 2016 campaign pledge to provide electricity 24 hours a day, saying it was harder to accomplish than he imagined.\n\nThe speech that marked the 216th anniversary of the world’s first black republic was originally slated to take place in the northern coastal town of Gonaives, where Jean-Jacques Dessalines declared Haiti’s independence. But the town, like many others, was hit by violent protests that began in September amid anger over corruption, fuel shortages and dwindling food supplies as opposition leaders and supporters demanded the resignation of Moïse. More than 40 people have been killed and dozens injured.\n\nLarge-scale protests in Port-au-Prince have since dissipated, although smaller ones are still occurring elsewhere in the country. On Wednesday, opposition leaders and supporters gathered in Gonaives to attend the funeral of an anti-government protester and then carried his coffin through the streets as more protesters joined them.\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.",4.0,MARTINSBURG WVa A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia police said\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County The Journal reported\n\nCody Wade Braithwaite 32 of Winchester Virginia was charged with DUI with death fleeing DUI and being a prohibited person in possession of a firearm and a fugitive from justice Berkeley County Sheriff Curtis Keller said\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop but the suspect’s vehicle sped up and hit a car which then struck a guardrail The driver of the car was pronounced dead at the scene The driver’s name was not immediately released\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped 

Transforming to lowercase

In [128]:
dfText['msg_lower1']= dfText['clean_msg1'].apply(lambda x: x.lower())
dfText['msg_lower2']= dfText['clean_msg2'].apply(lambda x: x.lower())
dfText.head(1)

,pair_id,doc1,doc2,Overall,clean_msg1,clean_msg2,msg_lower1,msg_lower2
0,1568535170_1558667669,"MARTINSBURG, W.Va. — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia, police said.\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County, The Journal reported.\n\nCody Wade Braithwaite, 32, of Winchester, Virginia, was charged with DUI with death, fleeing DUI, and being a prohibited person in possession of a firearm and a fugitive from justice, Berkeley County Sheriff Curtis Keller said.\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop, but the suspect’s vehicle sped up and hit a car, which then struck a guardrail. The driver of the car was pronounced dead at the scene. The driver’s name was not immediately released.\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped, Keller said.\n\nBraithwaite was being held without bond in the Eastern Regional Jail. It wasn’t immediately known whether Braithwaite has an attorney.\n\n___\n\nInformation from: The Journal, http://journal-news.net/\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.","PORT-AU-PRINCE, Haiti — Haitian President Jovenel Moïse broke with tradition on Wednesday and celebrated the country’s independence day in the capital for security reasons following months of political turmoil.\n\nMoïse, whose government has been accused of corruption, denounced graft during his speech at the National Palace in Port-au-Prince and urged Haiti’s elite to work with the government and help create employment.\n\n“We’re still extremely poor,” he said. “Those who continue to get rich find it normal that they do not pay taxes, find it normal that there can be no competition, find it normal that they set prices for consumers, especially when this consumer is the state itself.”\n\nMoïse also apologized for the country’s ongoing power outages and renewed his 2016 campaign pledge to provide electricity 24 hours a day, saying it was harder to accomplish than he imagined.\n\nThe speech that marked the 216th anniversary of the world’s first black republic was originally slated to take place in the northern coastal town of Gonaives, where Jean-Jacques Dessalines declared Haiti’s independence. But the town, like many others, was hit by violent protests that began in September amid anger over corruption, fuel shortages and dwindling food supplies as opposition leaders and supporters demanded the resignation of Moïse. More than 40 people have been killed and dozens injured.\n\nLarge-scale protests in Port-au-Prince have since dissipated, although smaller ones are still occurring elsewhere in the country. On Wednesday, opposition leaders and supporters gathered in Gonaives to attend the funeral of an anti-government protester and then carried his coffin through the streets as more protesters joined them.\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.",4.0,MARTINSBURG WVa A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia police said\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County The Journal reported\n\nCody Wade Braithwaite 32 of Winchester Virginia was charged with DUI with death fleeing DUI and being a prohibited person in possession of a firearm and a fugitive from justice Berkeley County Sheriff Curtis Keller said\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop but the suspect’s vehicle sped up and hit a car which then struck a guardrail The driver of the car was pronounced dead at the scene The driver’s name was not immediately released\n\nThe suspect’s vehicle also hit a deputy’s cruiser

Tokenization

In [129]:
dfText['msg_tokenized1']= dfText['msg_lower1'].apply(lambda x: nltk.word_tokenize(x))
dfText['msg_tokenized2']= dfText['msg_lower2'].apply(lambda x: nltk.word_tokenize(x))
dfText.head(1)

,pair_id,doc1,doc2,Overall,clean_msg1,clean_msg2,msg_lower1,msg_lower2,msg_tokenized1,msg_tokenized2
0,1568535170_1558667669,"MARTINSBURG, W.Va. — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia, police said.\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County, The Journal reported.\n\nCody Wade Braithwaite, 32, of Winchester, Virginia, was charged with DUI with death, fleeing DUI, and being a prohibited person in possession of a firearm and a fugitive from justice, Berkeley County Sheriff Curtis Keller said.\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop, but the suspect’s vehicle sped up and hit a car, which then struck a guardrail. The driver of the car was pronounced dead at the scene. The driver’s name was not immediately released.\n\nThe suspect’s vehicle also hit a deputy’s cruiser before being stopped, Keller said.\n\nBraithwaite was being held without bond in the Eastern Regional Jail. It wasn’t immediately known whether Braithwaite has an attorney.\n\n___\n\nInformation from: The Journal, http://journal-news.net/\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.","PORT-AU-PRINCE, Haiti — Haitian President Jovenel Moïse broke with tradition on Wednesday and celebrated the country’s independence day in the capital for security reasons following months of political turmoil.\n\nMoïse, whose government has been accused of corruption, denounced graft during his speech at the National Palace in Port-au-Prince and urged Haiti’s elite to work with the government and help create employment.\n\n“We’re still extremely poor,” he said. “Those who continue to get rich find it normal that they do not pay taxes, find it normal that there can be no competition, find it normal that they set prices for consumers, especially when this consumer is the state itself.”\n\nMoïse also apologized for the country’s ongoing power outages and renewed his 2016 campaign pledge to provide electricity 24 hours a day, saying it was harder to accomplish than he imagined.\n\nThe speech that marked the 216th anniversary of the world’s first black republic was originally slated to take place in the northern coastal town of Gonaives, where Jean-Jacques Dessalines declared Haiti’s independence. But the town, like many others, was hit by violent protests that began in September amid anger over corruption, fuel shortages and dwindling food supplies as opposition leaders and supporters demanded the resignation of Moïse. More than 40 people have been killed and dozens injured.\n\nLarge-scale protests in Port-au-Prince have since dissipated, although smaller ones are still occurring elsewhere in the country. On Wednesday, opposition leaders and supporters gathered in Gonaives to attend the funeral of an anti-government protester and then carried his coffin through the streets as more protesters joined them.\n\nCopyright 2020 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.",4.0,MARTINSBURG WVa A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia police said\n\nThe accident occurred early Wednesday along southbound Interstate 81 in Berkeley County The Journal reported\n\nCody Wade Braithwaite 32 of Winchester Virginia was charged with DUI with death fleeing DUI and being a prohibited person in possession of a firearm and a fugitive from justice Berkeley County Sheriff Curtis Keller said\n\nKeller said a deputy investigating calls about a reckless driver attempted to make a traffic stop but the suspect’s vehicle sped up and hit a car which then struck a guardrail The driver of the car was pronounced dead at the scene The driver’s name was not immediately released\n\nThe suspect’s vehic

Talvez fazer o sentence tokenizer:

Stopwords filtering

In [130]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.append('’')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [131]:
def removeStopwords(listWords):
    filteredWords = [word for word in listWords if word not in stop_words]
    return filteredWords

dfText['no_stopwords1']= dfText['msg_tokenized1'].apply(lambda x: removeStopwords(x))
dfText['no_stopwords2']= dfText['msg_tokenized2'].apply(lambda x: removeStopwords(x))
dfText[['msg_tokenized1', 'no_stopwords1', 'msg_tokenized2', 'no_stopwords2']].head(1)


,msg_tokenized1,no_stopwords1,msg_tokenized2,no_stopwords2
0,"[martinsburg, wva, a, suspected, drunken, driver, was, arrested, in, a, new, year, ’, s, day, highway, crash, that, killed, another, motorist, in, west, virginia, police, said, the, accident, occurred, early, wednesday, along, southbound, interstate, 81, in, berkeley, county, the, journal, reported, cody, wade, braithwaite, 32, of, winchester, virginia, was, charged, with, dui, with, death, fleeing, dui, and, being, a, prohibited, person, in, possession, of, a, firearm, and, a, fugitive, from, justice, berkeley, county, sheriff, curtis, keller, said, keller, said, a, deputy, investigating, calls, about, a, reckless, driver, attempted, to, make, a, traffic, stop, but, the, suspect, ’, s, vehicle, sped, ...]","[martinsburg, wva, suspected, drunken, driver, arrested, new, year, day, highway, crash, killed, another, motorist, west, virginia, police, said, accident, occurred, early, wednesday, along, southbound, interstate, 81, berkeley, county, journal, reported, cody, wade, braithwaite, 32, winchester, virginia, charged, dui, death, fleeing, dui, prohibited, person, possession, firearm, fugitive, justice, berkeley, county, sheriff, curtis, keller, said, keller, said, deputy, investigating, calls, reckless, driver, attempted, make, traffic, stop, suspect, vehicle, sped, hit, car, struck, guardrail, driver, car, pronounced, dead, scene, driver, name, immediately, released, suspect, vehicle, also, hit, deputy, cruiser, stopped, keller, said, braithwaite, held, without, bond, eastern, regional, jail, immediately, known, whether, braithwaite, ...]","[portauprince, haiti, haitian, president, jovenel, moïse, broke, with, tradition, on, wednesday, and, celebrated, the, country, ’, s, independence, day, in, the, capital, for, security, reasons, following, months, of, political, turmoil, moïse, whose, government, has, been, accused, of, corruption, denounced, graft, during, his, speech, at, the, national, palace, in, portauprince, and, urged, haiti, ’, s, elite, to, work, with, the, government, and, help, create, employment, we, ’, re, still, extremely, poor, he, said, those, who, continue, to, get, rich, find, it, normal, that, they, do, not, pay, taxes, find, it, normal, that, there, can, be, no, competition, find, it, normal, that, ...]","[portauprince, haiti, haitian, president, jovenel, moïse, broke, tradition, wednesday, celebrated, country, independence, day, capital, security, reasons, following, months, political, turmoil, moïse, whose, government, accused, corruption, denounced, graft, speech, national, palace, portauprince, urged, haiti, elite, work, government, help, create, employment, still, extremely, poor, said, continue, get, rich, find, normal, pay, taxes, find, normal, competition, find, normal, set, prices, consumers, especially, consumer, state, moïse, also, apologized, country, ongoing, power, outages, renewed, 2016, campaign, pledge, provide, electricity, 24, hours, day, saying, harder, accomplish, imagined, speech, marked, 216th, anniversary, world, first, black, republic, originally, slated, take, place, northern, coastal, town, gonaives, jeanjacques, dessalines, declared, ...]"


Stemming

In [132]:
#from nltk.stem.porter import PorterStemmer
#porter = PorterStemmer()

#def stemming(listWords):
#    stemText = [porter.stem(word) for word in listWords]
#    return stemText

#dfText['msg_stemmed1']= dfText['no_stopwords1'].apply(lambda x: stemming(x))
#dfText['msg_stemmed2']= dfText['no_stopwords2'].apply(lambda x: stemming(x))
#dfText[['no_stopwords1', 'msg_stemmed1', 'no_stopwords2', 'msg_stemmed2']].head(1)

Lemmatization

In [133]:
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(listWords):
    lemmText = [wordnet_lemmatizer.lemmatize(word) for word in listWords]
    return lemmText

dfText['msg_lemmatized1']= dfText['no_stopwords1'].apply(lambda x: lemmatizer(x))
dfText['msg_lemmatized2']= dfText['no_stopwords2'].apply(lambda x: lemmatizer(x))
dfText[['no_stopwords1', 'msg_lemmatized1']].head(1)

,no_stopwords1,msg_lemmatized1
0,"[martinsburg, wva, suspected, drunken, driver, arrested, new, year, day, highway, crash, killed, another, motorist, west, virginia, police, said, accident, occurred, early, wednesday, along, southbound, interstate, 81, berkeley, county, journal, reported, cody, wade, braithwaite, 32, winchester, virginia, charged, dui, death, fleeing, dui, prohibited, person, possession, firearm, fugitive, justice, berkeley, county, sheriff, curtis, keller, said, keller, said, deputy, investigating, calls, reckless, driver, attempted, make, traffic, stop, suspect, vehicle, sped, hit, car, struck, guardrail, driver, car, pronounced, dead, scene, driver, name, immediately, released, suspect, vehicle, also, hit, deputy, cruiser, stopped, keller, said, braithwaite, held, without, bond, eastern, regional, jail, immediately, known, whether, braithwaite, ...]","[martinsburg, wva, suspected, drunken, driver, arrested, new, year, day, highway, crash, killed, another, motorist, west, virginia, police, said, accident, occurred, early, wednesday, along, southbound, interstate, 81, berkeley, county, journal, reported, cody, wade, braithwaite, 32, winchester, virginia, charged, duo, death, fleeing, duo, prohibited, person, possession, firearm, fugitive, justice, berkeley, county, sheriff, curtis, keller, said, keller, said, deputy, investigating, call, reckless, driver, attempted, make, traffic, stop, suspect, vehicle, sped, hit, car, struck, guardrail, driver, car, pronounced, dead, scene, driver, name, immediately, released, suspect, vehicle, also, hit, deputy, cruiser, stopped, keller, said, braithwaite, held, without, bond, eastern, regional, jail, immediately, known, whether, braithwaite, ...]"


## Implementação dos algoritmos

### Jaccard

In [134]:
def calculate_jaccard(word_tokens1, word_tokens2):
	# Combine both tokens to find union.
	both_tokens = word_tokens1 + word_tokens2
	union = set(both_tokens)

	# Calculate intersection.
	intersection = set()
	for w in word_tokens1:
		if w in word_tokens2:
			intersection.add(w)

	if len(union)==0:
		jaccard_score = 0
	else:
		jaccard_score = len(intersection)/len(union)
	return jaccard_score

In [135]:
dfText['jaccard'] = dfText.apply(lambda row: calculate_jaccard(row['msg_lemmatized1'], row['msg_lemmatized2']), axis=1)
#calculate_jaccard(dfText['msg_lemmatized1'][0], dfText['msg_lemmatized2'][0])

In [136]:
dfText[dfText['jaccard']==0][['pair_id', 'msg_lemmatized1', 'msg_lemmatized2']]

,pair_id,msg_lemmatized1,msg_lemmatized2
9,1568535170_1558667669,"[0, 2, 4, 6, subject, truly, loyal, chief, magistrate, neither, advise, submit, arbitrary, measure]","[get, full, access, globeandmailcom, 199, per, week, first, 24, week, 199, per, week, first, 24, week]"
15,1568535170_1558667669,"[noticed, blocking, ad, keep, supporting, great, journalism, turning, ad, blocker, purchase, subscription, unlimited, access, real, news, count]","[portauprince, haiti, haitian, president, jovenel, moïse, broke, tradition, wednesday, celebrated, country, independence, day, capital, security, reason, following, month, political, turmoil, moïse, whose, government, accused, corruption, denounced, graft, speech, national, palace, portauprince, urged, haiti, elite, work, government, help, create, employment, still, extremely, poor, said, continue, get, rich, find, normal, pay, tax, find, normal, competition, find, normal, set, price, consumer, especially, consumer, state, moïse, also, apologized, country, ongoing, power, outage, renewed, 2016, campaign, pledge, provide, electricity, 24, hour, day, saying, harder, accomplish, imagined, speech, marked, 216th, anniversary, world, first, black, republic, originally, slated, take, place, northern, coastal, town, gonaives, jeanjacques, dessalines, declared, ...]"
20,1568535170_1558667669,"[fort, lewis, college, updating, dozen, panel, clock, tower, center, campus, better, recognize, past, way, thats, fully, inclusive, native, american, community, campus, family, directly, affected, indian, boarding, school]","[please, enable, javascript, watch, video, denver, cold, colorado, day, miguel, rivera, martin, garcia, roll, sleeve, get, work, two, teen, inspired, others, youtube, decided, make, channel, dedicated, cleaning, earth, called, 7, environment, make, feel, kind, good, cleaning, garcia, said, found, frustrating, bizarre, time, shattered, glass, bottle, rusted, scooter, thrown, lake, tire, covered, algae, plenty, can, around, rotella, park, one, first, video, rivera, say, one, annoying, find, broken, glass, bottle, soccer, field, north, denver, kid, kind, sitting, car, telling, u, like, thank, rivera, said, pair, hope, continue, make, video, inspire, people, clean, least, change, bad, habit]"
40,1568535170_1558667669,[],"[patient, suspected, virus, attended, surgery, phe, say, room, seen, used, cleaned, detergent, disinfectant, item, used, care, patient, bagged, clinical, waste, including, content, bin, quarantined, known, whether, patient, tested, positive, wncov, fabric, curtain, screen, also, bagged, infectious, linen, cloth, mop, head, used, cleaning, disposed, single, use, item, patient, spent, time, communal, area, cleaned, detergent, disinfectant, soon, possible, according, initial, guidance, gps, suspect, patient, could, virus, consultation, leave, room, immediately, leaving, patient, belonging, inside, wash, hand, conduct, remainder, consultation, phone, also, contact, local, health, protection, team, gps, suspect, patient, virus, telephone, consultation, avoid, facetoface, assessment, primary, care, instead, call, local, secondary, care, ...]"
62,1568535170_1558667669,"[annual, big, bo, classic, held, wolseywessington, wish, see]","[founder, general, overseer, mountain, fire, miracle, ministry, mfm, dr, daniel, olukoya, released, 31, prophecy, year, 2020, olukoya, tagged, 2020, ‘, year, revival, new, glory, urged, nigerian, pray, year, would, messy, confusing, according, many, leader, would, shifted, away, prophecy, 1, 2020, year, disobedience, god, would, yield, terrible, result, deliverance, reverse, 2, year, enemy, plan, bring, dog, spirit, strange, sexual, perversion, 3, year, round, clock, prayer, needed, national, restlessness, 4, year, impatience, would, lure, many, strange, marriage, 5, year, take, back, enemy, stolen, occupy, new, territory, 6, hot, prayer, required, 2020, 7, year, would, messy, confusing, 8, year, many, leader, would, shifted, ...]"
80,1568535170_1558667669,